In [1]:
!pip install python-dotenv
!pip install huggingface_hub

In [2]:
from huggingface_hub import login

login()

In [ ]:
# # If HF token is stored in .env:
# import os
# os.environ["HF_TOKEN"] = os.getenv('HF_TOKEN')

## Load the Model and Tokenizer from Local

In [ ]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM

# # Specify the path to your mer ged model directory
# merged_model_path = "merged_model"

# # Load the tokenizer
# tokenizer = AutoTokenizer.from_pretrained(merged_model_path)

# # Load the merged model
# model = AutoModelForCausalLM.from_pretrained(
#     merged_model_path,
#     torch_dtype=torch.float16,  # Adjust based on your saved model's dtype
#     device_map="auto"  # Automatically maps model to available devices
# )

# # If you have a GPU, ensure the model is on the GPU
# if torch.cuda.is_available():
#     model = model.cuda()

## Load the Model and Tokenizer from HF (after pushing)

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

username = "GuelGaMesh01"
model_id = f"{username}/gemma-2b-it-finetuned-mental-health-qa"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,  # Adjust based on your saved model's dtype
    device_map="auto"  # Automatically maps model to available devices
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/522 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

## Test the finetuned model

In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the fine-tuned model and tokenizer
finetuned_model_id = "GuelGaMesh01/gemma-2b-it-finetuned-mental-health-qa"
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Load the fine-tuned model and tokenizer from Hugging Face Hub
finetuned_tokenizer = AutoTokenizer.from_pretrained(finetuned_model_id)
finetuned_model = AutoModelForCausalLM.from_pretrained(finetuned_model_id).to(device)

# Helper functions to get the responses in the proper format
def get_completion(context: str, model, tokenizer) -> str:
    # Adjusted prompt template to fit the mental health counseling dataset format
    prompt_template = """
    <start_of_turn>User Input

    Below is a conversation between a user and a counselor. Write a response that appropriately completes the conversation.

    Context: {context}
    <end_of_turn>\n<start_of_turn>Model Response:
    """

    # Fill in the context into the prompt template
    prompt = prompt_template.format(context=context)

    # Tokenize the prompt
    encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

    # Move the inputs to the appropriate device (GPU)
    model_inputs = encodeds.to(device)

    # Generate the response
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=1000,  # Adjust max tokens if necessary
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode the generated response
    decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return decoded

# Example usage
context_example = "I am feeling very anxious about my upcoming exams. I haven't been sleeping well, and it's affecting my concentration. What should I do?"
response = get_completion(context_example, finetuned_model, finetuned_tokenizer)
print("Generated Response:", response)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Generated Response: 
    User Input

    Below is a conversation between a user and a counselor. Write a response that appropriately completes the conversation.
    
    Context: I am feeling very anxious about my upcoming exams. I haven't been sleeping well, and it's affecting my concentration. What should I do?
    
Model Response:
    There are a few things you can consider:
1) You may have an underlying medical condition that is contributing in the way you are feeling (e.g., allergies, anxiety disorders, depression);  yung as that's the thing at the heart of your feelings. Please schedule an appointment with your doctor (or family doctor); 
2) Anxiety disorders can be very useful to help you in a way. I may be able to help you manage your anxiety through cognitive behavioral therapy (CBT).   
3) Also, look back over your life before the stressful part of school started. What were your accomplishments? Was there anything you did and thought was great? Reflecting on these moments can

In [6]:
query = """\n\n Please provide a thoughtful response to the following mental health context. Write an analysis of the user's feelings in between <analysis></analysis>.
           Then provide a suggested course of action or response in between <answer></answer>. Here is the context:

           Context: A young adult reports feeling overwhelmed by stress at work and at home. They feel like they can't manage their responsibilities and have trouble sleeping.
           They also mention feeling increasingly irritable with their loved ones and are unsure how to manage these emotions."""

# Pass 'query' as the 'context' parameter in the get_completion function
result = get_completion(context=query, model=finetuned_model, tokenizer=finetuned_tokenizer)
print(result)



    User Input

    Below is a conversation between a user and a counselor. Write a response that appropriately completes the conversation.
    
    Context: 

 Please provide a thoughtful response to the following mental health context. Write an analysis of the user's feelings in between <analysis></analysis>.
           Then provide a suggested course of action or response in between <answer></answer>. Here is the context:

           Context: A young adult reports feeling overwhelmed by stress at work and at home. They feel like they can't manage their responsibilities and have trouble sleeping.
           They also mention feeling increasingly irritable with their loved ones and are unsure how to manage these emotions.
    
Model Response:
    	Within the context you provided, there are various factors that are contributing to the feelings a young adult is having. 
	In regards to them feeling overwhelmed at work and their entire household, and feeling as though they have little co

In [7]:
print(f"{result.split('Analysis:')[-1]}")


    User Input

    Below is a conversation between a user and a counselor. Write a response that appropriately completes the conversation.
    
    Context: 

 Please provide a thoughtful response to the following mental health context. Write an analysis of the user's feelings in between <analysis></analysis>.
           Then provide a suggested course of action or response in between <answer></answer>. Here is the context:

           Context: A young adult reports feeling overwhelmed by stress at work and at home. They feel like they can't manage their responsibilities and have trouble sleeping.
           They also mention feeling increasingly irritable with their loved ones and are unsure how to manage these emotions.
    
Model Response:
    	Within the context you provided, there are various factors that are contributing to the feelings a young adult is having. 
	In regards to them feeling overwhelmed at work and their entire household, and feeling as though they have little co